#### Import

In [1]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [2]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10")
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

<Transform:
<Orientation: 
array([[-1.13445515e-04,  8.65996364e-01, -5.00050283e-01],
       [ 9.99999991e-01,  1.33176672e-04,  3.76962167e-06],
       [ 6.98595113e-05, -5.00050278e-01, -8.65996371e-01]])>
<Vector: (-0.51000, 0.39997, 0.59058)>
>


In [3]:
from ddh import DDGripper
from scooping_primitives import HighSpeedScooping

In [4]:
gripper = DDGripper("ddh_scooping")
scoop = HighSpeedScooping(ur, gripper, "HSS")

reading gripper config...
connecting to odrive...
found left finger
found right fingers
reading scooping config...


In [5]:
scoop.initialize_gripper_pose((-0.65,0.4,180))

Setting left tip: (147.5, 22.00961894323342)
Setting right tip: (120.0, -0.3589838486224579)
Setting pose: 
[-0.635       0.4         0.4        -1.92687451 -1.92687451  0.51630447]


#### testing

In [ ]:
gripper.disarm()

In [ ]:
gripper.arm(200)

In [ ]:
gripper.arm(150)
gripper.set_parallel_jaw(10, 0)
time.sleep(0.5)
a2_init = (gripper.left_a2 + gripper.right_a2)/2 

In [ ]:
gripper.set_stiffness(50)

In [ ]:
gripper.set_left_a1_phi(-5, 20)
gripper.set_right_a1_phi(5, -20)

In [ ]:
gripper.set_left_a1_phi(-0, 20)
gripper.set_right_a1_phi(0, -20)

In [ ]:
print(gripper.left_a2,gripper.right_a2)

In [ ]:
# check finger collision
while 1:
    a2_cur = (gripper.left_a2 + gripper.right_a2)/2
#     print(a2_cur)
    if a2_cur - a2_init > 0.5:
        print ("Collision detected!")
        break

In [ ]:
print(ur.getl())

In [ ]:
init = (-0.73, 0.17, 0.4, pi, 0, 0)
ur.movel(init, tcp_acc, tcp_vel)

In [ ]:
smack_vel = 100
smack_acc = 0.3
stop_acc = 0.5 #0.5
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)

In [ ]:
ur.speedl([0,0,smack_vel,0,0,0],stop_acc,5)
time.sleep(0.5)
ur.stopl(stop_acc)